  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=02ece6678b7fd19d715e29cd654b1f5f00fff21d571372fc9f9404ef8d5b09c7
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [ ]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
from efficientnet_pytorch import EfficientNet
import os
from PIL import Image
import pandas as pd

input_folder_path_train = "/content/drive/MyDrive/images/preprocessed/train"
output_folder_path_train = "/content/drive/MyDrive/images/preprocessed/train_cropped"
input_folder_path_test = "/content/drive/MyDrive/images/preprocessed/test"
output_folder_path_test = "/content/drive/MyDrive/images/preprocessed/test_cropped"
new_width = 255  # Set the desired width
new_height = 255  # Set the desired height


In [ ]:
def resize_and_break(input_folder, output_folder, new_width, new_height):
    # Create output folder and subfolders(classes) if they doesn't exist
    outputs = ['class_1','class_2','class_3','class_4','class_5','class_6','class_7','class_8']
    if not os.path.exists(output_folder):
            os.makedirs(output_folder)
    for i in outputs:
        folder_path = os.path.join(output_folder, i)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    # Loop through all files in the input folder
    if input_folder == '/content/drive/MyDrive/images/preprocessed/train':
        csv_train = os.path.join(input_folder, "CSAW-M_train.csv")
        df_label = pd.read_csv(csv_train,sep = ';')
    else:
        csv_test = os.path.join(input_folder, "CSAW-M_test.csv")
        df_label = pd.read_csv(csv_test,sep = ';')


    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.gif')):  # Add more extensions if needed
            # Grab label for file
            label = df_label.loc[df_label['Filename'] == filename, 'Label'].values[0]

            # Assign output folder based on label
            folder_path = os.path.join(os.path.join(output_folder, 'class_'+str(label)))

            # Construct full file paths
            input_path = os.path.join(input_folder, filename)

            # Open the image
            image = Image.open(input_path)

            # Resize the image
            resized_image = image.resize((new_width, new_height), Image.LANCZOS)

            # Get the file name (without extension) from the input path
            file_name = os.path.splitext(filename)[0]

            # Save the resized image to the correct label folder
            output_path = os.path.join(folder_path, f"{file_name}_resized.jpg")
            resized_image.save(output_path)

In [ ]:
# Uncomment to resize and label data
resize_and_break(input_folder_path_train, output_folder_path_train, new_width, new_height)
resize_and_break(input_folder_path_test, output_folder_path_test, new_width, new_height)


In [ ]:
# Define your dataset and dataloaders
data_transform = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
])

# Replace 'your_dataset_directory' with the path to your dataset
train_dataset = datasets.ImageFolder(root=output_folder_path_train, transform=data_transform)
val_dataset = datasets.ImageFolder(root=output_folder_path_test, transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


#from torchvision.models import efficientnet_b4, EfficientNet_b4_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

#efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
#efficientnet_b0(weights="DEFAULT")

# Instantiate the model
print(f"Is Cuda supported: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
model = models.convnext_tiny(weights="DEFAULT")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5  # You can increase this number for better training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Save the trained model
torch.save(model.state_dict(), 'efficientnetb4_model.pth')

Is Cuda supported: True


Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-7eb33cd5.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 432MB/s]


Epoch 1/5, Loss: 1.9678809550784577
Validation Accuracy: 25.15%
Epoch 2/5, Loss: 1.7571620633138105
Validation Accuracy: 38.43%
Epoch 3/5, Loss: 1.6561600445900988
Validation Accuracy: 36.02%
Epoch 4/5, Loss: 1.5101421883442259
Validation Accuracy: 36.42%
Epoch 5/5, Loss: 1.2680543105074222
Validation Accuracy: 29.58%
